In [ ]:
import cv2

video_capture = cv2.VideoCapture('ваш_видеофайл.mp4')

frame_count = 0

# Чтение видео по кадрам
while True:
    ret, frame = video_capture.read()

    if not ret:
        break

    filename = f'frame_{frame_count:04d}.jpg'  # Имя файла для кадра
    cv2.imwrite(filename, frame)

    frame_count += 1

video_capture.release()

print(f'Преобразовано {frame_count} кадров')


In [ ]:
from PIL import Image

def encode_lsb_with_markers(image_path, message, output_path):
    image = Image.open(image_path)
    width, height = image.size

    binary_message = "1010101010101111" + ''.join(format(ord(char), '016b') for char in message) + "0101010101010111"

    if len(binary_message) > width * height * 3:
        print("Слишком длинное сообщение для данного изображения.")
        return

    encoded_image = image.copy()
    data_index = 0

    for y in range(height):
        for x in range(width):
            pixel = list(image.getpixel((x, y)))
            for color_channel in range(3):
                if data_index < len(binary_message):
                    pixel[color_channel] = int(bin(pixel[color_channel])[:-1] + binary_message[data_index], 2)
                    data_index += 1
            encoded_image.putpixel((x, y), tuple(pixel))

    encoded_image.save(output_path)
    print("Сообщение успешно закодировано и сохранено.")

In [ ]:
def decode_lsb_with_markers(encoded_image_path):
    encoded_image = Image.open(encoded_image_path)
    width, height = encoded_image.size
    binary_message = ""

    for y in range(height):
        for x in range(width):
            pixel = encoded_image.getpixel((x, y))
            for color_channel in range(3):
                binary_message += bin(pixel[color_channel])[-1]

    marker_start = "1010101010101111"
    marker_end = "0101010101010111"
    start_index = binary_message.find(marker_start)
    end_index = binary_message.find(marker_end, start_index)

    if start_index != -1 and end_index != -1:
        binary_message = binary_message[start_index+len(marker_start):end_index]
        decoded_message = ''.join(chr(int(binary_message[i:i+16], 2)) for i in range(0, len(binary_message), 16))
        print("Скрытое сообщение:", decoded_message)
    else:
        print("Маркеры не найдены.")

In [ ]:
image_path = "input_image.png"
message = """
test text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 

test text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 
Hellj 7^5/345 Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 
Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТtest text 12345 Hellj 7^5/* Текст ТЕСТ
test text 3
"""
encoded_image_path = "encoded_image.png"

encode_lsb_with_markers(image_path, message, encoded_image_path)

In [ ]:
encoded_image_path = "encoded_image.png"
decode_lsb_with_markers(encoded_image_path)